## 1. Подготовка

### 1.1 Поднятие постгрес БД в докере

>> docker run --name sber_test -e POSTGRES_PASSWORD=pswrd99 -p 5432:5432 -d postgres

Данная команда создаст контейнер под названием sber_test с портом 5432.


### 1.2 Подключение к базе данных в программе DBeaver
Данные для подключения к БД в программе DBeaver следующие:
* Host: localhost
* Port: 5432
* Database: postgres
* Username: postgres
* Password: pswrd99

### 1.3 Создание таблицы с данными

In [1]:
import psycopg2 as pg

connection = pg.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="pswrd99"
)
cursor = connection.cursor()

In [2]:
cursor.execute("""
DROP TABLE IF EXISTS t ;
CREATE TABLE IF NOT EXISTS t (
	key SERIAL PRIMARY KEY,
	id int,
	phone varchar,
	mail varchar
);
""")
connection.commit()

### 1.4 Заполнение данными
Код также записан в файле `insert_data.sql`

In [3]:
cursor.execute("""
INSERT INTO t (id, phone, mail)
VALUES 
(12345,'89997776655','test@mail.ru'),
(54321,'87778885566','two@mail.ru'),
(98765,'87776664577','three@mail.ru'),
(66678,'87778885566','four@mail.ru'),
(34567,'84547895566','four@mail.ru'),
(34567,'89087545678','five@mail.ru');
""")
connection.commit()

## 2. Задания

На основании заданного поля (это может быть id, phone, mail) получить все "связанные данные"

Например:
если задать поиск по условию phone = 87778885566;

Результат должен быть следующим:
```
2; 54321;87778885566;two@mail.ru
4; 66678;87778885566;four@mail.ru
5; 34567;84547895566;four@mail.ru
6; 34567;89087545678;five@mail.ru
```
Задание следует выполнить на python и на SQL (по сути два задания). Диалект SQL можно использовать любой.

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Решение на SQL

Как мне здесь видится, чтобы вытащить все связанные данные, нам в любом случае нужно прибегать к некоторому либо циклу, либо хардкоду.
Потому что как только мы получили определенные данные, нам нужно проверить, а не пересекаются ли теперь новые данные с данными из общей таблицы.
Так что даже если прибегнуть к CROSS JOIN и перевязать все данные со всеми, то чтобы получить последнее значение с ключом 6, нам сначала нужно понять, что данные с ключом 5 тоже к нам относятся. Поэтому в запросе просто хардкод с джойнами. Надеюсь я рассуждал верно.

In [3]:
q = '''
WITH RECURSIVE r AS (
SELECT "key", id, phone, mail
FROM t 
WHERE phone = '87778885566'
UNION
SELECT t."key", t.id, t.phone, t.mail 
FROM t 
JOIN r 
    ON 
        t."key" = r."key" OR
        t.id = r.id       OR
        t.phone = r.phone OR 
        t.mail = r.mail
    
)
SELECT * FROM r;
'''
task = pd.read_sql_query(q, connection)
task

,key,id,phone,mail
0,2,54321,87778885566,two@mail.ru
1,4,66678,87778885566,four@mail.ru
2,5,34567,84547895566,four@mail.ru
3,6,34567,89087545678,five@mail.ru


### Решение на Python

Здесь можно уже решить проблему циклом и выставить необходимое количество итераций. Например у нас большая выборка, поэтому может мы захотим и 10 раз пройтись по данным.

In [4]:
query = "SELECT * FROM t"
data = pd.read_sql_query(query, connection)
data

,key,id,phone,mail
0,1,12345,89997776655,test@mail.ru
1,2,54321,87778885566,two@mail.ru
2,3,98765,87776664577,three@mail.ru
3,4,66678,87778885566,four@mail.ru
4,5,34567,84547895566,four@mail.ru
5,6,34567,89087545678,five@mail.ru


In [5]:
spec_phone_data = data.query("phone=='87778885566'")
spec_phone_data

,key,id,phone,mail
1,2,54321,87778885566,two@mail.ru
3,4,66678,87778885566,four@mail.ru


In [6]:
def get_all_bounded_data(start_data, full_data, n_iters: int = 5):
    for i in range(5):  # Проходимся несколько раз по столбцам...
        for col in full_data.columns[1:]:  # [id, phone, mail]
            col_data = start_data[col].to_list()  # вытаскиваем все данные определенного столбца, которые у нас есть сейчас
            new_data = full_data[full_data[col].isin(col_data)]  # и ищем все совпадения в основной таблице
            start_data = pd.concat([start_data, new_data])  # после присоеднияем полученный запрос к стартовому,
                                                            # тем самым каждый раз добавляя новые данные в стартовую таблицу

    return start_data.drop_duplicates('key').reset_index(drop=True)  # в конце избавляемся от дубликатов по ключу key

In [7]:
get_all_bounded_data(spec_phone_data, data, 5)

,key,id,phone,mail
0,2,54321,87778885566,two@mail.ru
1,4,66678,87778885566,four@mail.ru
2,5,34567,84547895566,four@mail.ru
3,6,34567,89087545678,five@mail.ru


## 3. Задание 2

Договор, оформленный клиентом у нас впервые, будем называть первым договором; договор, оформленный после – вторым; далее – третьим; и так далее.
Необходимо написать SQL запрос к базе для представления его результатов в сводной таблице вида:
```
Количество 1 договоров, оформленных в 2020; Количество 2 договоров, оформленных в 2020; Количество 3 договоров, оформленных в 2020...
Мужчины
Женщины
```

### 3.1 Заполним данные

In [48]:
# Create tables
q = """
    DROP TABLE IF EXISTS loans_table;
    CREATE TABLE IF NOT EXISTS loans_table (
    LOAN_ID int,
    CLIENT_ID int,
    LOAN_DATE date,
    LOAN_AMOUNT float);

    DROP TABLE IF EXISTS clients_table;
    CREATE TABLE IF NOT EXISTS clients_table (
    CLIENT_ID int,
    CLIENT_NAME VARCHAR(20),
    BIRTHDAY date,
    GENDER VARCHAR(20));
"""
cursor.execute(q)
connection.commit()

In [49]:
# Populate Clients
cursor.execute(
"""
    INSERT INTO clients_table (CLIENT_ID, CLIENT_NAME, BIRTHDAY, GENDER)
    VALUES 
    (1, 'Иванов Иван', '1985-01-12', 'Мужской'),
    (2, 'Петрова Александра', '1990-05-25', 'Женский'),
    (3, 'Смирнов Максим', '1978-09-03', 'Мужской'),
    (4, 'Кузнецов Дмитрий', '1982-07-17', 'Мужской'),
    (5, 'Васильева Елизавета', '1995-11-29', 'Женский'),
    (6, 'Михайлов Алексей', '1976-04-08', 'Мужской'),
    (7, 'Новикова Анастасия', '1988-06-06', 'Женский'),
    (8, 'Козлов Сергей', '1992-02-21', 'Мужской'),
    (9, 'Алексеева Мария', '1984-12-14', 'Женский'),
    (10, 'Егоров Артем', '1991-10-19', 'Мужской')
"""
)
connection.commit()

In [8]:
import random
from datetime import datetime, timedelta
import numpy as np

In [9]:
def daterange(date1, date2, day_interval):
    date_range = int((date2 - date1).days)
    for n in range(0, date_range+1, day_interval):
        yield date1 + timedelta(days=n)

In [10]:
def populate_loans_table(day_interval: int = 7):
    """Генерирует рандомные данные и заполняет ими таблицу с договорами"""
    mean = 5
    std = 2
    for i, loan_date in enumerate(daterange(datetime.fromisoformat('2020-01-01'), datetime.fromisoformat('2020-12-31'), day_interval)):
        loan_id = i + 1
        client_id = int(np.random.normal(mean, std))
        loan_amount = round(random.uniform(1000, 5000), 2)
        
        sql_query = "INSERT INTO loans_table (LOAN_ID, CLIENT_ID, LOAN_DATE, LOAN_AMOUNT) VALUES (%s, %s, %s, %s)"
        values = (loan_id, client_id, loan_date, loan_amount)
        
        cursor.execute(sql_query, values)
    connection.commit()

In [53]:
populate_loans_table()

### 3.2 Решение

### Методом SQL

Хардкодный метод

In [117]:
pt = pd.read_sql_query("""
    SELECT 
        gender as "Пол",
        COUNT(CASE WHEN num_loans >= 1 THEN 1 END) AS "Количество 1 договоров, оформленных в 2020",
        COUNT(CASE WHEN num_loans >= 2 THEN 1 END) AS "Количество 2 договоров, оформленных в 2020",
        COUNT(CASE WHEN num_loans >= 3 THEN 1 END) AS "Количество 3 договоров, оформленных в 2020",
        COUNT(CASE WHEN num_loans >= 4 THEN 1 END) AS "Количество 4 договоров, оформленных в 2020",
        COUNT(CASE WHEN num_loans >= 5 THEN 1 END) AS "Количество 5 договоров, оформленных в 2020"
    FROM (
        SELECT                  -- отберем клиентов и посчитаем по каждому, сколько вышло у них подписать договоров в этом году
            lt.client_id,
            ct.gender,
            COUNT(*) AS num_loans
        FROM loans_table lt
        JOIN clients_table ct ON lt.client_id = ct.client_id
        WHERE date_part('year', lt.loan_date) < 2021  -- обрубаем данные за год, больше 2020
        GROUP BY lt.client_id, ct.gender
        HAVING date_part('year', MIN(lt.loan_date)) = 2020  -- Проверяем, чтобы дата первого договора пользователя была не раньше 2020 года
                                -- потому что иначе может быть человек с договорами на 2019 год и мы его посчитаем, как его первый договор
    ) t
    GROUP BY "Пол";
""", connection)
pt

,Пол,"Количество 1 договоров, оформленны","Количество 2 договоров, оформленны","Количество 3 договоров, оформленны","Количество 4 договоров, оформленны","Количество 5 договоров, оформленны"
0,Женский,3,3,3,2,2
1,Мужской,5,4,4,3,3


Динамический метод

In [11]:
q = """
DO $$
DECLARE
    column_list text := '';
    _query text;
    _cursor CONSTANT refcursor := '_cursor';
BEGIN
    with loans as (
        select lt.client_id, gender, COUNT(loan_id) AS num_loans
        from loans_table lt
        join clients_table ct on lt.client_id = ct.client_id
        where EXTRACT(year from loan_date) = 2020
        group by lt.client_id, gender
    ), gen_rows as (
        select * from generate_series(1, (select max(num_loans) from loans)) n_loans
    ), clients_by_loans as (
        select n_loans, gender, COUNT(case when loans.num_loans >= n_loans then 1 end) cnt
        from gen_rows, loans
        group by 1, 2
        order by 1, 2
    )
    -- Формируем список столбцов на основе уникальных значений столбца n_loans
    SELECT string_agg('"Количество ' || n_loans || 'х договоров" int', ', ')
    INTO column_list
    FROM (select DISTINCT n_loans from clients_by_loans order by 1) t;
    
    -- Формируем запрос для получения кросс таблицы
    _query := format('
        SELECT * FROM crosstab(
           ''select gender, n_loans, cnt from temp_table order by 1, 2'',
           ''select distinct n_loans::int from temp_table order by 1 asc''
        ) AS ct (
          "Пол" text,
          %s
        )', column_list);
    OPEN _cursor FOR EXECUTE _query;
END
$$;
FETCH ALL FROM _cursor;
"""
pt = pd.read_sql_query(q, connection)
pt

,Пол,Количество 1х договоров,Количество 2х договоров,Количество 3х договоров,Количество 4х договоров,Количество 5х договоров,Количество 6х договоров,Количество 7х договоров,Количество 8х договоров,Количество 9х договоров,Количество 10х договоров,Количество 11х договоров,Количество 12х договоров,Количество 13х договоров,Количество 14х договоров,Количество 15х договоров,Количество 16х договоров
0,Женский,3,3,3,2,2,1,1,1,1,0,0,0,0,0,0,0
1,Мужской,5,4,4,3,3,3,2,2,2,2,1,1,1,1,1,1


### Методом Python

In [12]:
loans = pd.read_sql_query("""
    SELECT *, date_part('year', loan_date)::int as year
    FROM loans_table
    JOIN clients_table
    USING (client_id)
""", connection)
loans.head()

,client_id,loan_id,loan_date,loan_amount,client_name,birthday,gender,year
0,6,1,2020-01-01,3117.85,Михайлов Алексей,1976-04-08,Мужской,2020
1,4,2,2020-01-08,3642.66,Кузнецов Дмитрий,1982-07-17,Мужской,2020
2,5,3,2020-01-15,3070.86,Васильева Елизавета,1995-11-29,Женский,2020
3,4,4,2020-01-22,3859.52,Кузнецов Дмитрий,1982-07-17,Мужской,2020
4,3,5,2020-01-29,1170.61,Смирнов Максим,1978-09-03,Мужской,2020


In [16]:
def get_count_of_loans(loans: pd.DataFrame):
    loans_count = loans.groupby(['client_id', 'gender']).agg({'loan_id': 'count'})
    t = loans_count[loans_count >= 1].groupby('gender').count()
    for i in range(2, loans_count['loan_id'].max() + 1):
        ti = loans_count[loans_count >= i].groupby('gender').count()
        t = pd.concat([t, ti], axis=1)
    t.columns = [f"Количество {j} договоров" for j in range(1, loans_count['loan_id'].max()+1)]
    return t

get_count_of_loans(loans)

,Количество 1 договоров,Количество 2 договоров,Количество 3 договоров,Количество 4 договоров,Количество 5 договоров,Количество 6 договоров,Количество 7 договоров,Количество 8 договоров,Количество 9 договоров,Количество 10 договоров,Количество 11 договоров,Количество 12 договоров,Количество 13 договоров,Количество 14 договоров,Количество 15 договоров,Количество 16 договоров
gender,,,,,,,,,,,,,,,,
Женский,3,3,3,2,2,1,1,1,1,0,0,0,0,0,0,0
Мужской,5,4,4,3,3,3,2,2,2,2,1,1,1,1,1,1
